[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/hernanescu/mtg-bookarizer/blob/main/mtg-bookarizer.ipynb)

In [1]:
# librerías que tenemos que instalar
!pip install ebooklib
!pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.6/111.6 kB 2.6 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 70.8 MB/s eta 0:00:00:00:0100:01
  Created wheel for ebooklib: filename=EbookLib-0.17.1-py3-none-any.whl size=38184 sha256=08a59471268f6b0d866466f95cc61a2fcd266d172fb1f1e70a0e79c6c55928c3
  Stored in directory: /home/jupyter/.cache/pip/wheels/43/39/fd/db4f652431a55d28472ba7f5f7c9a8efad03b97f443a48ea2f
Successfully built ebooklib


In [2]:
# scrapeo del sitio y procesamiento
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
import numpy as np
import nltk
import matplotlib as plt
import seaborn as sns

In [3]:
# función para scrapear el nombre de quien haya escrito la historia
def find_author(URL):
    request = requests.get(URL)
    soup = BeautifulSoup(request.text, 'html.parser')
    cuerpo = soup.find('div', {'class':'author'})
    autor = cuerpo.find_all('p')
    parrafos = [p.text.replace("<h1>","").replace("</h1>","") for p in autor]
    full_name = parrafos[0].replace('By ', '')
    return full_name

In [4]:
# función para scrapear el título de la historia
def find_title(URL):
    request = requests.get(URL)
    soup = BeautifulSoup(request.text, 'html.parser')
    cuerpo = soup.find(id='main-content')
    titulo = cuerpo.find_all('h1')
    parrafos = [p.text.replace("<h1>","").replace("</h1>","") for p in titulo]
    name = parrafos[0]
    return name

In [5]:
# lo picante: función para levantar el texto de la historia, preservando los atributos estéticos y líneas divisoras
# una vez que está el texto, para poder meterlo en el ebook tiene que ser un único string con formato HTML
def get_text(link):
    # Vamos al link
    r = requests.get(link)
    # print(r.text)
    soup = BeautifulSoup(r.text, 'html.parser')
    # print(soup)
    # Buscamos los parrafos
    cuerpo = soup.find(id='content-detail-page-of-an-article')
    parrafos = cuerpo.find_all(['p', 'hr'])
    # Agarramos solo el texto de cada parrafo y le sacamos \r y \n que definen nuevas lineas
    
    init = '<p>'
    close = '</p>'
    res = ''

    # generación de string único 
    for ele in parrafos:
        res += init + str(ele) + close
  
    return res

In [6]:
# en este caso tomamos la historia de Dominaria United! 
# decidí ordenarlos en orden cronológico, para intercalar las side stories :)

LINK_1 = 'https://magic.wizards.com/en/articles/archive/magic-story/episode-1-echoes-dark-2022-08-10'
LINK_2 = 'https://magic.wizards.com/en/articles/archive/magic-story/episode-2-sand-hourglass-2022-08-11'
LINK_3 = 'https://magic.wizards.com/en/articles/archive/magic-story/episode-3-locked-tower-2022-08-12'
LINK_4 = 'https://magic.wizards.com/en/articles/archive/magic-story/homecoming-2022-08-12'
LINK_5 = 'https://magic.wizards.com/en/articles/archive/magic-story/episode-4-brutal-blow-2022-08-15'
LINK_6 = 'https://magic.wizards.com/en/articles/archive/magic-story/education-ulf-2022-08-15'
LINK_7 = 'https://magic.wizards.com/en/articles/archive/magic-story/death-and-salvation-2022-08-16'
LINK_8 = 'https://magic.wizards.com/en/articles/archive/magic-story/shards-nightmares-2022-08-17'
LINK_9 = 'https://magic.wizards.com/en/articles/archive/magic-story/faith-birds-2022-08-17'
LINK_10 = 'https://magic.wizards.com/en/articles/archive/magic-story/episode-5-whisper-wind-2022-08-18'


In [7]:
# buscamos el autor de cada link
AUTHOR_1 = find_author(LINK_1)
AUTHOR_2 = find_author(LINK_2)
AUTHOR_3 = find_author(LINK_3)
AUTHOR_4 = find_author(LINK_4)
AUTHOR_5 = find_author(LINK_5)
AUTHOR_6 = find_author(LINK_6)
AUTHOR_7 = find_author(LINK_7)
AUTHOR_8 = find_author(LINK_8)
AUTHOR_9 = find_author(LINK_9)
AUTHOR_10 = find_author(LINK_10)

In [8]:
# buscamos el título de cada link
TITLE_1 = find_title(LINK_1)
TITLE_2 = find_title(LINK_2)
TITLE_3 = find_title(LINK_3)
TITLE_4 = find_title(LINK_4)
TITLE_5 = find_title(LINK_5)
TITLE_6 = find_title(LINK_6)
TITLE_7 = find_title(LINK_7)
TITLE_8 = find_title(LINK_8)
TITLE_9 = find_title(LINK_9)
TITLE_10 = find_title(LINK_10)


In [9]:
# buscamos el texto de cada link
STORY_1 = get_text(LINK_1)
STORY_2 = get_text(LINK_2)
STORY_3 = get_text(LINK_3)
STORY_4 = get_text(LINK_4)
STORY_5 = get_text(LINK_5)
STORY_6 = get_text(LINK_6)
STORY_7 = get_text(LINK_7)
STORY_8 = get_text(LINK_8)
STORY_9 = get_text(LINK_9)
STORY_10 = get_text(LINK_10)


In [13]:
# pequeña prueba
STORY_10[0:100]

"<p><p>Teferi slammed a Phyrexian monstrosity onto Karn's worktop and pinned it with a knife. The cre"

In [21]:
# generación del ebook
from ebooklib import epub

# primero se instancia un objeto "book"
book = epub.EpubBook()
TITLE = 'Magic: The Gathering: Dominaria United'
book.set_identifier('herno-mtg-0-1')
book.set_title(TITLE)
book.set_language('en')

# se le agregan los autores
book.add_author(AUTHOR_1)
book.add_author(AUTHOR_2)
book.add_author(AUTHOR_3)
book.add_author(AUTHOR_4)
book.add_author(AUTHOR_5)
book.add_author(AUTHOR_6)
book.add_author(AUTHOR_7)
book.add_author(AUTHOR_8)
book.add_author(AUTHOR_9)
book.add_author(AUTHOR_10)

# se agregan los capítulos (título y archivo interno de renderizado)
c1 = epub.EpubHtml(title=TITLE_1, file_name='chap_01.xhtml', lang='en')
c2 = epub.EpubHtml(title=TITLE_2, file_name='chap_02.xhtml', lang='en')
c3 = epub.EpubHtml(title=TITLE_3, file_name='chap_03.xhtml', lang='en')
c4 = epub.EpubHtml(title=TITLE_4, file_name='chap_04.xhtml', lang='en')
c5 = epub.EpubHtml(title=TITLE_5, file_name='chap_05.xhtml', lang='en')
c6 = epub.EpubHtml(title=TITLE_6, file_name='chap_06.xhtml', lang='en')
c7 = epub.EpubHtml(title=TITLE_7, file_name='chap_07.xhtml', lang='en')
c8 = epub.EpubHtml(title=TITLE_8, file_name='chap_08.xhtml', lang='en')
c9 = epub.EpubHtml(title=TITLE_9, file_name='chap_09.xhtml', lang='en')
c10 = epub.EpubHtml(title=TITLE_10, file_name='chap_10.xhtml', lang='en')

# a cada capítulo se le agrega su contenido. 
# Para que quede más lindo en la lectura, a ese string le agregamos un h1 con el título del capítulo
c1.content=f'<h1>{TITLE_1}</h1>{STORY_1}'
c2.content=f'<h1>{TITLE_2}</h1>{STORY_2}'
c3.content=f'<h1>{TITLE_3}</h1>{STORY_3}'
c4.content=f'<h1>{TITLE_4}</h1>{STORY_4}'
c5.content=f'<h1>{TITLE_5}</h1>{STORY_5}'
c6.content=f'<h1>{TITLE_6}</h1>{STORY_6}'
c7.content=f'<h1>{TITLE_7}</h1>{STORY_7}'
c8.content=f'<h1>{TITLE_8}</h1>{STORY_8}'
c9.content=f'<h1>{TITLE_9}</h1>{STORY_9}'
c10.content=f'<h1>{TITLE_10}</h1>{STORY_10}'

# le agregamos cada capítulo al objeto book
book.add_item(c1)
book.add_item(c2)
book.add_item(c3)
book.add_item(c4)
book.add_item(c5)
book.add_item(c6)
book.add_item(c7)
book.add_item(c8)
book.add_item(c9)
book.add_item(c10)


# definimos la tabla de contenidos
book.toc = (epub.Link('chap_01.xhtml', TITLE_1, TITLE_1),
            epub.Link('chap_02.xhtml', TITLE_2, TITLE_2),
            epub.Link('chap_03.xhtml', TITLE_3, TITLE_3),
            epub.Link('chap_04.xhtml', TITLE_4, TITLE_4),
            epub.Link('chap_05.xhtml', TITLE_5, TITLE_5),
            epub.Link('chap_06.xhtml', TITLE_6, TITLE_6),
            epub.Link('chap_07.xhtml', TITLE_7, TITLE_7),
            epub.Link('chap_08.xhtml', TITLE_8, TITLE_8),
            epub.Link('chap_09.xhtml', TITLE_9, TITLE_9),
            epub.Link('chap_10.xhtml', TITLE_10, TITLE_10)
            )

# hay que agregar archivos ncv y nav, son de configuraciones internas
book.add_item(epub.EpubNcx())
book.add_item(epub.EpubNav())

# definimos un estilo CSS, este es el default
style = 'BODY {color: white;}'
nav_css = epub.EpubItem(uid="style_nav", file_name="style/nav.css", media_type="text/css", content=style)

# agregamos el CSS
book.add_item(nav_css)

# se arma el "lomo" del libro
book.spine = ['nav', c1, c2, c3, c4, c5, c6, c7, c8, c9, c10]

# se escribe
epub.write_epub('mtg-dominaria-united.epub', book, {})
